In [2]:
#logistic regression

import torch
import numpy as np

# python 读取csv https://www.cnblogs.com/cloud-ken/p/8432999.html
# https://blog.csdn.net/jiaowosiye/article/details/80782729?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2
xy = np.loadtxt('/Users/dyq/Desktop/pytorch_liu/diabetes.csv', delimiter=',', dtype=np.float32) 
x_data = torch.from_numpy(xy[:,:-1])
y_data = torch.from_numpy(xy[:, [-1]])
print(x_data.shape)
print(y_data.shape)




OSError: /Users/dyq/Desktop/pytorch_liu/diabetes.csv not found.

In [6]:
#https://www.bilibili.com/video/BV1Y7411d7Ys?p=12
import torch
import numpy as np

input_size = 4
hidden_size = 4
num_layers = 1
batch_size = 1
seq_len = 5

#hello
idx2char = ['e','h','l','o']
one_hot_lookup = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]

#x_data = [1,0,2,2,3] # hello
x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2] # ohlol

x_one_hot = [one_hot_lookup[x] for x in x_data]
print(x_one_hot)
print(torch.Tensor(x_one_hot).shape)

inputs = torch.Tensor(x_one_hot).view(seq_len, batch_size,input_size)
print(inputs)
print(inputs.shape)

labels = torch.LongTensor(y_data)
print(labels)



[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
torch.Size([5, 4])
tensor([[[0., 1., 0., 0.]],

        [[1., 0., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 1.]]])
torch.Size([5, 1, 4])
tensor([3, 1, 2, 3, 2])


In [ ]:


class CNN_Model(torch.nn.Module):
    def __init__(self,input_size, hidden_size, batch_size, num_layer = 1):
        super(CNN_Model,self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size = self.input_size,
                               hidden_size = self.hidden_size,
                               num_layers = num_layers)
    def forward(self,input):
        hidden = torch.zeros(self.num_layers,self.batch_size,self.hidden_size)
        out , out1 = self.rnn(input,hidden)
        return (out.view(-1,self.hidden_size),out1)
        # seqlen * batch_size , hidden_size
        # 因为以后cross entropy的时候 ， labels 会变成 (seqlen * batch_size , 1)
        
CNN_net = CNN_Model(input_size, hidden_size, batch_size, num_layers)


CNN_net = CNN_Model(input_size , hidden_size , batch_size, num_layers)
criterion = torch.nn.CrossEntropyLoss()
#soft_max + cross_entropy
optimizer = torch.optim.Adam(CNN_net.parameters() , lr = 0.05)

for epoch in range(15):
    optimizer.zero_grad()  
    outputs,outputs1 = CNN_net(inputs)
    print(outputs.shape)
    print(outputs[-1])
    
    print(outputs1.shape)
    print(outputs1[0][0])
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    
    _,idx = outputs.max(dim = 1)
    idx= idx.data.numpy()
    print("predicted: ", "".join([idx2char[x] for x in idx]), end= "")
    print(",Epoch [%d/15] loss = %.3f" %(epoch +1 ,loss.item()))